# About 
This scripts removes as much shared peptides as possible with trivial methods.


In [1]:

import os 
import re
import pandas as pd
import numpy as np

In [3]:
# read in .fasta file and count shared peptides

os.chdir("/home/ptruong/git/dia_sum")

fasta = "UP000000625_ecoli.fasta"

filename = "database/2021-06-16/" + fasta
#filename = "database/napedro_3mixed_human_yeast_ecoli_20140403_iRT_reverse.fasta"


file = open(filename, "r")


protein_list = []
sequence_list = []
for line in file: 
    if line[0] == ">":
        protein = line 
    else:
        sequence = line.rstrip()
        split_sequence = re.split(r"(?<=[KR])", sequence)
        split_sequence = list(dict.fromkeys(split_sequence))
        sequence_list += split_sequence
        protein_list += [protein for i in range(len(split_sequence))]
        

df_ = pd.DataFrame(np.array([protein_list, sequence_list]).T, columns = ["protein", "sequence"])
df = pd.DataFrame(np.array([protein_list, sequence_list]).T, columns = ["protein", "sequence"])


In [4]:
def decoy_map(protein):
    if protein.split("_")[0] == ">reverse":
        return True
    else:
        return False
    
    
df["decoy"] = df.protein.map(decoy_map)

df = df[df.decoy == False]
df["seq_length"] = df.sequence.str.len()
df = df[df["seq_length"] > 7]
df.drop("seq_length", axis = 1, inplace = True)
df.drop_duplicates(inplace=True)


# Find protein with only one sequence

In [5]:
counted_df = df.groupby("sequence").count().sort_values(by = "protein", ascending = False)
counted_df

,protein,decoy
sequence,,
NQLAMLFTLANLFR,12,12
EELAEVDVDWL,12,12
DPEMHQTK,12,12
WLAEAGVMMTQGTLVDATIIEAPSSTK,12,12
IHCMQHWYNLSDGAMEDALYEIASMR,12,12
...,...,...
GVMVVEER,1,1
GVMVVGVHFMSLELGGR,1,1
GVNAIELMHDAIGHILQLR,1,1


In [6]:
single_protein_seq = counted_df[counted_df.protein == 1]

# investigate if a single protein sequence has proteins containing single sequences.

In [7]:
df_single_seq = df[df.sequence.isin(single_protein_seq.index)]

In [8]:
df_single_seq.groupby("sequence").count().max()

protein    1
decoy      1
dtype: int64

In [9]:
# seems fine, keep these proteins 
len(df_single_seq.protein.unique())

4306

In [10]:
len(df.protein.unique())

4374

In [11]:
keep_list = df_single_seq.protein.unique()

# Read in both fasta files and compare results.

In [13]:
os.chdir("/home/ptruong/git/dia_sum")
filename = "database/2021-06-16/" + fasta
#filename = "database/napedro_3mixed_human_yeast_ecoli_20140403_iRT_reverse.fasta"
file = open(filename, "r")

file_w = open(f"database/2021-06-16/no_shared_{fasta}", "w")

for line in file:
    if line[0] == ">":
        protein = line 
        file_w.write(protein)
    else:
        if protein in keep_list:
            sequence = line
            file_w.write(sequence)
        

'>sp|Q9ULT0|TTC7A_HUMAN Tetratricopeptide repeat protein 7A OS=Homo sapiens OX=9606 GN=TTC7A PE=1 SV=3\n'